# 경로 확인

In [1]:
%cd /content/drive/MyDrive/02.git/healthcare-hackathon
print('===')
!ls

/content/drive/MyDrive/02.git/healthcare-hackathon
===
backend  docker-compose.yml  frontend  langchain  model  README.md  temp.py


In [2]:
%cd model/fer2013-finetuning
!ls

/content/drive/MyDrive/02.git/healthcare-hackathon/model/fer2013-finetuning
configs			       MobileNet_v4_extra-train.ipynb  README.md
docs			       MobileNet_v4_init_train.ipynb   requirements.txt
MobileNet_model_structure.pth  models			       utils


# MobileNet 불러오기
- save로 모델 생성
- 이전에 학습했던 가중치 파일 적용

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/02.git/healthcare-hackathon/model/fer2013-finetuning')

In [4]:
import torch
from models.classifier import create_emotion_classifier
from utils.dataset import create_dataloaders
from utils.trainer import EmotionTrainer
from configs.config import Config

# 모델 생성
model = create_emotion_classifier(
    num_classes=Config.NUM_CLASSES,
    backbone_name=Config.MODEL_NAME
)

torch.save(model, 'MobileNet_model_structure.pth')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
!cp MobileNet_model_structure.pth ../fer2013-test/MobileNet_model_structure.pth

- 가중치 파일 구드 링크
- https://drive.google.com/file/d/1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_/view?usp=drive_link

In [9]:
# 가중치 파일 다운로드(구글드라이브 링크)
file_id = "1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_"

# gdown을 사용하여 파일 다운로드
!gdown {file_id} -O /content/MobileNet_finetuned.pth

Downloading...
From (original): https://drive.google.com/uc?id=1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_
From (redirected): https://drive.google.com/uc?id=1eGZ8Kb3jd_V9wYSI7BBgbxKrItcuvGg_&confirm=t&uuid=6243b4eb-b98b-41b3-9cec-93ebb052ba52
To: /content/MobileNet_finetuned.pth
100% 37.1M/37.1M [00:00<00:00, 115MB/s]


In [10]:
# 저장된 가중치 로드
checkpoint = torch.load('/content/MobileNet_finetuned.pth', map_location=Config.DEVICE)
model.load_state_dict(checkpoint)
print("✅ best_model_copy.pth 로드 완료")

# 모델의 디바이스 확인
model.to(Config.DEVICE)
device = next(model.parameters()).device
print(f"모델은 현재 {device}에 있습니다.")

✅ best_model_copy.pth 로드 완료
모델은 현재 cpu에 있습니다.


# 데이터셋 준비
- https://drive.google.com/file/d/1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT/view?usp=drive_link

In [10]:
file_id = "1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT"

# gdown을 사용하여 파일 다운로드
!gdown {file_id} -O /content/fer2013.zip

# 압축 해제
!unzip -q /content/fer2013.zip -d /content/fer2013

Downloading...
From (original): https://drive.google.com/uc?id=1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT
From (redirected): https://drive.google.com/uc?id=1OOFyCEGTbFrsuYr1Q5Ds7QAvpiXkqyhT&confirm=t&uuid=7223d7a6-c134-491f-8674-cabf602725d0
To: /content/fer2013.zip
100% 63.3M/63.3M [00:00<00:00, 92.0MB/s]
replace /content/fer2013/test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [11]:
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm


# 테스트 데이터셋 로드
train_loader, val_loader = create_dataloaders(
    data_dir='/content/fer2013',
    batch_size=256,
    num_workers=2
)

# 모델을 평가 모드로 전환
model.eval()

# 예측 및 실제 레이블 저장
all_preds = []
all_labels = []

# 테스트 데이터셋에 대해 예측 수행
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):  # tqdm으로 반복문 감싸기
        inputs, labels = inputs.to(Config.DEVICE), labels.to(Config.DEVICE)

        # 모델 예측
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # 예측 및 실제 레이블 저장
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 정확도 계산
accuracy = accuracy_score(all_labels, all_preds)
print(f"테스트 정확도: {accuracy * 100:.2f}%")

# 클래스 레이블 (예: 0부터 6까지의 7가지 클래스)
class_labels = [0, 1, 2, 3, 4, 5, 6]

# 분류 보고서 출력
report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in class_labels])
print(report)

Evaluating:   0%|          | 0/29 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Evaluating:   0%|          | 0/29 [00:47<?, ?it/s]


KeyboardInterrupt: 

In [6]:
!pip install -U -q transformers

In [7]:
from transformers import pipeline

# 사전 학습된 모델을 사용하여 이미지 분류 파이프라인 생성
pipe = pipeline("image-classification", model="trpakov/vit-face-expression")

from transformers import AutoImageProcessor, AutoModelForImageClassification

# 사전 학습된 모델에 맞는 이미지 전처리기 로드
processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression", use_fast=True)

# 사전 학습된 이미지 분류 모델 로드
model = AutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

# 모델을 지정한 디바이스(CPU 또는 GPU)로 이동
model.to(Config.DEVICE)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [5]:
# 테스트 데이터셋 로드(is_norm=False, [0,1] 로 정규화)
train_loader, val_loader = create_dataloaders(
    data_dir='/content/fer2013',
    batch_size=256,
    num_workers=2,
    is_norm=False
)

In [12]:
# 모델을 평가 모드로 전환
model.eval()

# 예측 및 실제 레이블 저장
all_preds = []
all_labels = []

# 테스트 데이터셋에 대해 예측 수행
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        # 이미지 전처리
        inputs = processor(images=inputs, return_tensors="pt").to(Config.DEVICE)

        # 모델 예측
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        # 예측 및 실제 레이블 저장
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 정확도 계산
accuracy = accuracy_score(all_labels, all_preds)
print(f"테스트 정확도: {accuracy * 100:.2f}%")

# 클래스 레이블 (예: 0부터 6까지의 7가지 클래스)
class_labels = [0, 1, 2, 3, 4, 5, 6]

# 분류 보고서 출력
report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in class_labels])
print(report)

Evaluating:   0%|          | 0/29 [01:48<?, ?it/s]


KeyboardInterrupt: 